# Topic detection

Dataset used: quotes from the *New York Times* from 2019.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyLDAvis
import pyLDAvis.gensim_models

from src.data_loader import download_test_data
from src.df_factory import (add_col_qid, create_df_joined_quotes,
                            create_df_speaker, create_df_test)
from src.sentiment_analysis import add_col_compound_score
from src.text_processing import (add_bigrams, add_col_bow, add_col_tokens,
                                 create_dictionary, get_lda_model)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/quentin/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/home/quentin/.local/lib/python3.8/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
/home/quentin/.local/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
# Download data
download_test_data()

Filename quotes-2019-nytimes.json.bz2 already exists


In [4]:
# Create the test dataframe
df = create_df_test()

# Add QID column
add_col_qid(df)

df

100%|██████████| 207527/207527 [00:00<00:00, 1412395.87it/s]


,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,qid
quoteID,,,,,,,,,
2019-04-17-024782,"It is not a low-income immigration,",James Fisher,"[Q16213953, Q20707104, Q43143598, Q58886302, Q...",2019-04-17 13:31:18,1,"[[James Fisher, 0.7475], [None, 0.2525]]",[https://www.nytimes.com/2019/04/17/realestate...,E,Q16213953
2019-04-02-001128,a champion figure skater switching to roller s...,John Updike,[Q105756],2019-04-02 14:58:33,2,"[[John Updike, 0.5856], [None, 0.4144]]",[https://www.nytimes.com/2019/04/02/opinion/vl...,E,Q105756
2019-05-09-055187,It makes it much more difficult for him to mak...,None,[],2019-05-09 18:11:29,1,"[[None, 0.6493], [President Bill Clinton, 0.27...",[http://mobile.nytimes.com/2019/05/09/world/as...,E,None
2019-10-31-056366,"It puts me in a predicament,",Xavier Becerra,[Q1855840],2019-10-31 16:45:15,3,"[[Xavier Becerra, 0.9065], [None, 0.0909], [St...",[http://www.nytimes.com/2019/10/31/technology/...,E,Q1855840
2019-01-04-001792,A Pile of Leaves.,None,[],2019-01-04 10:00:07,1,"[[None, 0.8737], [Jason Fulford, 0.1263]]",[https://www.nytimes.com/2019/01/04/books/revi...,E,None
...,...,...,...,...,...,...,...,...,...
2019-07-11-031126,I put in one of those wire shelves that you bu...,None,[],2019-07-11 04:00:15,1,"[[None, 0.8794], [Marissa Paternoster, 0.1206]]",[https://www.nytimes.com/2019/07/11/style/band...,E,None
2019-06-25-032409,I used to go there religiously and buy anythin...,Carlos Santana,"[Q2367902, Q5042623, Q819016]",2019-06-25 19:52:47,2,"[[Carlos Santana, 0.9484], [None, 0.04], [Rick...",[https://www.nytimes.com/2019/06/25/arts/music...,E,Q2367902
2019-05-02-043481,I was the welterweight boxing champ at St. Mik...,Leonard Patrick Kelly,[Q7143620],2019-05-02 21:41:51,3,"[[Leonard Patrick Kelly, 0.5743], [None, 0.388...",[http://nytimes.com/2019/05/02/obituaries/red-...,E,Q7143620


In [5]:
# QIDS from Wikidata
qids = {
    'Donald Trump': 'Q22686',
    'Joe Biden': 'Q6279',
    'Barack Obama': 'Q76',
    'Hillary Clinton': 'Q6294',
}

# Create Trump's dataframe
df_trump = create_df_speaker(df, qids['Donald Trump'])
df_trump

,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,qid
quoteID,,,,,,,,,
2019-02-15-009152,Asylum seekers in Mexico face a heightened ris...,President Donald Trump,[Q22686],2019-02-15 03:20:00,5,"[[President Donald Trump, 0.7752], [None, 0.22...",[http://www.abajournal.com/news/article/lawsui...,E,Q22686
2019-05-13-016076,"Don't forget they're a member of NATO, and a v...",President Donald Trump,[Q22686],2019-05-13 00:00:00,68,"[[President Donald Trump, 0.6683], [None, 0.23...",[http://kazu.org/post/trump-greets-hungarys-ha...,E,Q22686
2019-08-25-016871,I think it's a positive. Other people agree wi...,President Donald Trump,[Q22686],2019-08-25 23:19:22,7,"[[President Donald Trump, 0.6994], [None, 0.15...",[http://www.nytimes.com/2019/08/25/world/europ...,E,Q22686
2019-08-31-024043,"In other words, they're running badly and they...",President Donald Trump,[Q22686],2019-08-31 00:21:27,2,"[[President Donald Trump, 0.7363], [None, 0.26...",[http://www.nytimes.com/2019/08/30/us/politics...,E,Q22686
2019-05-09-051472,"In typical fashion, as soon as Trump Park was ...",President Trump,[Q22686],2019-05-09 09:25:21,1,"[[President Trump, 0.6722], [None, 0.3037], [D...",[https://www.nytimes.com/2019/05/09/nyregion/n...,E,Q22686
...,...,...,...,...,...,...,...,...,...
2019-11-19-032738,"I had a feeling that it was coming because, yo...",President Trump,[Q22686],2019-11-19 22:54:47,1,"[[President Trump, 0.7344], [None, 0.2656]]",[http://nytimes.com/2019/11/19/us/navy-seals-e...,E,Q22686
2019-04-04-038327,"I have heard that she was nasty to me, but she...",President Donald Trump,[Q22686],2019-04-04 00:00:00,42,"[[President Donald Trump, 0.5646], [Donald Tru...",[http://feeds.foxnews.com/~r/foxnews/politics/...,E,Q22686
2019-02-06-052048,investments in the cutting-edge industries of ...,President Donald Trump,[Q22686],2019-02-06 22:34:01,40,"[[President Donald Trump, 0.5853], [None, 0.23...",[https://defence.pk/pdf/threads/trump-preparin...,E,Q22686


In [6]:
# Add compound score
add_col_compound_score(df_trump)
df_trump

100%|██████████| 4557/4557 [00:00<00:00, 8293.12it/s]


,quotation,speaker,qids,date,numOccurrences,probas,urls,phase,qid,compound_score
quoteID,,,,,,,,,,
2019-02-15-009152,Asylum seekers in Mexico face a heightened ris...,President Donald Trump,[Q22686],2019-02-15 03:20:00,5,"[[President Donald Trump, 0.7752], [None, 0.22...",[http://www.abajournal.com/news/article/lawsui...,E,Q22686,-0.9300
2019-05-13-016076,"Don't forget they're a member of NATO, and a v...",President Donald Trump,[Q22686],2019-05-13 00:00:00,68,"[[President Donald Trump, 0.6683], [None, 0.23...",[http://kazu.org/post/trump-greets-hungarys-ha...,E,Q22686,0.5939
2019-08-25-016871,I think it's a positive. Other people agree wi...,President Donald Trump,[Q22686],2019-08-25 23:19:22,7,"[[President Donald Trump, 0.6994], [None, 0.15...",[http://www.nytimes.com/2019/08/25/world/europ...,E,Q22686,0.8225
2019-08-31-024043,"In other words, they're running badly and they...",President Donald Trump,[Q22686],2019-08-31 00:21:27,2,"[[President Donald Trump, 0.7363], [None, 0.26...",[http://www.nytimes.com/2019/08/30/us/politics...,E,Q22686,-0.9100
2019-05-09-051472,"In typical fashion, as soon as Trump Park was ...",President Trump,[Q22686],2019-05-09 09:25:21,1,"[[President Trump, 0.6722], [None, 0.3037], [D...",[https://www.nytimes.com/2019/05/09/nyregion/n...,E,Q22686,-0.3400
...,...,...,...,...,...,...,...,...,...,...
2019-11-19-032738,"I had a feeling that it was coming because, yo...",President Trump,[Q22686],2019-11-19 22:54:47,1,"[[President Trump, 0.7344], [None, 0.2656]]",[http://nytimes.com/2019/11/19/us/navy-seals-e...,E,Q22686,0.1280
2019-04-04-038327,"I have heard that she was nasty to me, but she...",President Donald Trump,[Q22686],2019-04-04 00:00:00,42,"[[President Donald Trump, 0.5646], [Donald Tru...",[http://feeds.foxnews.com/~r/foxnews/politics/...,E,Q22686,-0.3182
2019-02-06-052048,investments in the cutting-edge industries of ...,President Donald Trump,[Q22686],2019-02-06 22:34:01,40,"[[President Donald Trump, 0.5853], [None, 0.23...",[https://defence.pk/pdf/threads/trump-preparin...,E,Q22686,0.0000


In [7]:
# Create dataframe with joined quotes by speaker
df_joined_quotes = create_df_joined_quotes(df)
df_joined_quotes

100%|██████████| 26162/26162 [00:00<00:00, 86366.76it/s]


,qid,quotation
0,Q1000491,"No way will something like that happen again,"
1,Q100071,"It was like, `I did everything right, and I've..."
2,Q1000791,"I had 45 turkeys on my front lawn, I walked st..."
3,Q1000820,I use them to get my frustrations out by telli...
4,Q1001244,"Max, I'm going to stop it. Max, you're getting..."
...,...,...
26157,Q996054,"The one-man, one-gun, one-bullet is not what o..."
26158,Q997091,She was ready to move to a nice place that was...
26159,Q997312,private reserves in which thought has develope...
26160,Q997380,He was a hero-like horse for me. I was sincere...


In [8]:
# Select four politicians
df_politicians = df_joined_quotes[df_joined_quotes.qid.isin(qids.values())]
df_politicians

,qid,quotation
8874,Q22686,Asylum seekers in Mexico face a heightened ris...
22394,Q6279,"He's not a fossil fuel executive, It's disgust..."
22442,Q6294,What is it you're finding out? Where does it l...
24891,Q76,"Tell her to keep her mouth shut, Elizabeth was..."


In [9]:
# Lengths of quotations
df_politicians.quotation.apply(len)

8874     376056
22394     61688
22442      7239
24891     15955
Name: quotation, dtype: int64

In [10]:
# Tokenize
add_col_tokens(df_politicians)
df_politicians

100%|██████████| 4/4 [00:20<00:00,  5.14s/it]


,qid,quotation,tokens
8874,Q22686,Asylum seekers in Mexico face a heightened ris...,"[asylum, seeker, Mexico, face, heightened, ris..."
22394,Q6279,"He's not a fossil fuel executive, It's disgust...","[fossil, fuel, executive, disgusting, ask, que..."
22442,Q6294,What is it you're finding out? Where does it l...,"[find, lead, lead, conclusion, president, comm..."
24891,Q76,"Tell her to keep her mouth shut, Elizabeth was...","[tell, mouth, shut, Elizabeth, sound, alarm, p..."


In [11]:
# Lengths of list of tokens
df_politicians.tokens.apply(len)

8874     26676
22394     4258
22442      508
24891     1150
Name: tokens, dtype: int64

In [12]:
# Add bigrams
add_bigrams(df_politicians)
df_politicians

100%|██████████| 4/4 [00:00<00:00, 93.69it/s]


,qid,quotation,tokens
8874,Q22686,Asylum seekers in Mexico face a heightened ris...,"[asylum, seeker, Mexico, face, heightened, ris..."
22394,Q6279,"He's not a fossil fuel executive, It's disgust...","[fossil, fuel, executive, disgusting, ask, que..."
22442,Q6294,What is it you're finding out? Where does it l...,"[find, lead, lead, conclusion, president, comm..."
24891,Q76,"Tell her to keep her mouth shut, Elizabeth was...","[tell, mouth, shut, Elizabeth, sound, alarm, p..."


In [13]:
# Lengths of list of tokens
df_politicians.tokens.apply(len)

8874     27059
22394     4310
22442      514
24891     1152
Name: tokens, dtype: int64

In [14]:
# Create dictionary
dictionary = create_dictionary(df_politicians, min_wordcount=0, max_freq=1)
print(dictionary)

# Add Bag-of-words column
add_col_bow(df_politicians, dictionary)
df_politicians

Dictionary(5409 unique tokens: ['ABT', 'AIDS', 'ALABAMA', 'AMERICA', 'AMERICAN']...)


100%|██████████| 4/4 [00:00<00:00, 404.50it/s]


,qid,quotation,tokens,bow
8874,Q22686,Asylum seekers in Mexico face a heightened ris...,"[asylum, seeker, Mexico, face, heightened, ris...","[(0, 1), (1, 1), (2, 1), (3, 5), (4, 1), (5, 1..."
22394,Q6279,"He's not a fossil fuel executive, It's disgust...","[fossil, fuel, executive, disgusting, ask, que...","[(12, 4), (21, 1), (32, 3), (33, 17), (34, 1),..."
22442,Q6294,What is it you're finding out? Where does it l...,"[find, lead, lead, conclusion, president, comm...","[(33, 1), (117, 1), (134, 1), (173, 1), (174, ..."
24891,Q76,"Tell her to keep her mouth shut, Elizabeth was...","[tell, mouth, shut, Elizabeth, sound, alarm, p...","[(20, 1), (33, 1), (34, 3), (35, 1), (84, 2), ..."


In [15]:
# Create LDA model
lda_model = get_lda_model(df_politicians, dictionary)
lda_model.show_topics(num_topics=10, num_words=4)

[(75, '0.009*"people" + 0.008*"think" + 0.008*"want" + 0.007*"know"'),
 (48, '0.011*"think" + 0.011*"people" + 0.008*"country" + 0.007*"good"'),
 (24, '0.012*"know" + 0.009*"think" + 0.008*"people" + 0.008*"want"'),
 (84, '0.008*"people" + 0.008*"want" + 0.007*"like" + 0.007*"know"'),
 (47, '0.013*"people" + 0.011*"know" + 0.010*"like" + 0.010*"think"'),
 (27, '0.012*"people" + 0.010*"know" + 0.009*"country" + 0.009*"like"'),
 (60, '0.013*"people" + 0.011*"think" + 0.008*"like" + 0.008*"know"'),
 (65, '0.010*"people" + 0.009*"think" + 0.009*"know" + 0.008*"like"'),
 (72, '0.012*"know" + 0.011*"people" + 0.010*"want" + 0.010*"think"'),
 (73, '0.009*"people" + 0.008*"know" + 0.008*"like" + 0.007*"great"')]

In [16]:
# Display topics
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(
    topic_model=lda_model,
    corpus=df_politicians.bow,
    dictionary=dictionary,
)

/home/quentin/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
62     0.003282  0.005354       1        1  65.894534
11     0.010137 -0.005993       2        1  13.576675
78     0.003797  0.011128       3        1  11.005464
88     0.015272  0.007367       4        1   7.957197
15     0.014784  0.000513       5        1   1.187666
...         ...       ...     ...      ...        ...
32    -0.003752  0.002471      96        1   0.000139
31    -0.002268 -0.001441      97        1   0.000139
30     0.000034  0.003086      98        1   0.000139
29    -0.002679  0.003493      99        1   0.000139
50    -0.008110  0.002395     100        1   0.000139

[100 rows x 5 columns], topic_info=           Term        Freq       Total  Category  logprob  loglift
3434     people  400.000000  400.000000   Default  30.0000  30.0000
4367      think  382.000000  382.000000   Default  29.0000  29.0000
3019       like  359.000000  359.000000   Default  28.0000  28.0000
2947       know  355.000000  355.000000   Default  27.0000  27.0000
4638       want  284.000000  284.000000   Default  26.0000  26.0000
...         ...         ...         ...       ...      ...      ...
3002        let    0.000112   80.253442  Topic100  -6.0202  -0.0001
4386       time    0.000193  225.764275  Topic100  -5.4725  -0.4867
4737       year    0.000156  158.501701  Topic100  -5.6893  -0.3497
1376        bad    0.000135  123.991528  Topic100  -5.8316  -0.2465
3572  president    0.000131  176.632899  Topic100  -5.8645  -0.6333

[8327 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         1  0.847147   ALABAMA
4         1  0.891019  AMERICAN
5         1  0.818330     ANGRY
6         1  0.778442    ANTIFA
14        1  0.780801      Adam
...     ...       ...       ...
4739      1  0.816908      yell
4744      1  0.566996     young
4744      2  0.261690     young
4744      3  0.087230     young
4744      4  0.043615     young

[3079 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[63, 12, 79, 89, 16, 100, 58, 66, 73, 54, 72, 71, 70, 69, 68, 67, 65, 59, 55, 64, 75, 56, 57, 62, 61, 60, 74, 78, 76, 90, 99, 98, 97, 96, 95, 94, 93, 92, 91, 88, 77, 87, 86, 85, 84, 83, 82, 81, 80, 52, 53, 1, 2, 50, 25, 24, 23, 22, 21, 20, 19, 18, 17, 15, 14, 13, 11, 10, 9, 8, 7, 6, 5, 4, 3, 26, 27, 28, 40, 49, 48, 47, 46, 45, 44, 43, 42, 41, 39, 29, 38, 37, 36, 35, 34, 33, 32, 31, 30, 51])